In [ ]:
from llama_index.core import SimpleDirectoryReader
 
# load documents
documents = SimpleDirectoryReader(input_files=["project_proposal.pdf"]).load_data()

In [ ]:
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core import Settings
from llama_index.llms.azure_openai import AzureOpenAI
from llama_index.embeddings.azure_openai import AzureOpenAIEmbedding
import os
from dotenv import load_dotenv

load_dotenv()

splitter = SentenceSplitter(chunk_size=1024)
nodes = splitter.get_nodes_from_documents(documents)

Settings.llm = AzureOpenAI(model="gpt-4o",
                            deployment_name="gpt-4o",
                            api_key=os.environ.get('AZURE_OPENAI_API_KEY'),
                            azure_endpoint="https://softtech-openai-ailab.openai.azure.com/",
                            api_version="2024-02-01")
Settings.embed_model = AzureOpenAIEmbedding(
    model="text-embedding-ada-002",
    api_key=os.environ.get('AZURE_OPENAI_API_KEY'),
    deployment_name='text-embedding-ada-002',
    azure_endpoint="https://softtech-openai-ailab.openai.azure.com/",
    api_version="2024-02-01",
)

In [ ]:
from llama_index.core import SummaryIndex, VectorStoreIndex

summary_index = SummaryIndex(nodes, show_progress=True)
vector_index = VectorStoreIndex(nodes, insert_batch_size=48, show_progress=True)

In [ ]:
summary_query_engine = summary_index.as_query_engine(
    response_mode="tree_summarize",
    use_async=True,
)
vector_query_engine = vector_index.as_query_engine()

In [ ]:
from llama_index.core.tools import QueryEngineTool


summary_tool = QueryEngineTool.from_defaults(
    query_engine=summary_query_engine,
    description=(
        "Useful for summarization questions related to paper"
    ),
)

vector_tool = QueryEngineTool.from_defaults(
    query_engine=vector_query_engine,
    description=(
        "Useful for retrieving specific context from the paper."
    ),
)

In [ ]:
from llama_index.core.query_engine.router_query_engine import RouterQueryEngine
from llama_index.core.selectors import LLMSingleSelector


query_engine = RouterQueryEngine(
    selector=LLMSingleSelector.from_defaults(),
    query_engine_tools=[
        summary_tool,
        vector_tool,
    ],
    verbose=True
)

response = query_engine.query("What is the summary of the document?")
print(str(response))